# Erwin Antepuesto

## Instructions:

1. Read the Bernoulli Mixture Model Derivation.
2. Read about Stochastic Expectation-Maximization (EM) Algorithm: https://www.sciencedirect.com/science/article/pii/S0167947320302504.
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.
4. Use the data from the paper: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.
6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.
7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes.

### Datasets Import

In [23]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 
from kmodes.kmodes import KModes
from sklearn.metrics import fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import fowlkes_mallows_score
from sklearn.preprocessing import LabelEncoder
from scipy.special import logsumexp

In [24]:
# Datasets
import pandas as pd

data = {
    'Data set': ['Soybean', 'Zoo', 'Heart disease', 'Breast cancer', 'Dermatology', 'Letters(E,F)', 'DNA', 'Mushroom', 'Iris', 'Isolet', 'COIL20', 'OpticalDigits', 'PenDigits'],
    'Type': ['Categorical', 'Categorical', 'Categorical', 'Categorical', 'Categorical', 'Categorical', 'Categorical', 'Categorical', 'Numerical', 'Numerical', 'Numerical', 'Numerical', 'Numerical'],
    'Availability': ['Available', 'Available', 'Available', 'Available', 'Available', 'Not Available', 'Not Available', 'Available', 'Available', 'Not Available', 'Not Available', 'Not Available', 'Not Available']
}

df = pd.DataFrame(data)
df

,Data set,Type,Availability
0,Soybean,Categorical,Available
1,Zoo,Categorical,Available
2,Heart disease,Categorical,Available
3,Breast cancer,Categorical,Available
4,Dermatology,Categorical,Available
5,"Letters(E,F)",Categorical,Not Available
6,DNA,Categorical,Not Available
7,Mushroom,Categorical,Available
8,Iris,Numerical,Available
9,Isolet,Numerical,Not Available


In [25]:
# Datasets Import
# Soybean
soybean_small = fetch_ucirepo(id=91)
sbf = soybean_small.data.features
sbt = soybean_small.data.targets
soybean_df = pd.merge(sbf, sbt, left_index=True, right_index=True)

# Zoo
zoo = fetch_ucirepo(id=111)
zg = zoo.data.features
zt = zoo.data.targets
zoo_df = pd.merge(zg, zt, left_index=True, right_index=True)

# Heart Disease
heart_disease = fetch_ucirepo(id=45)
hdg = heart_disease.data.features
hdt = heart_disease.data.targets
heart_disease_df = pd.merge(hdg, hdt, left_index=True, right_index=True)

# Breast Cancer
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)
bcf = breast_cancer_wisconsin_original.data.features
bct = breast_cancer_wisconsin_original.data.targets
breast_cancer_df = pd.merge(bcf, bct, left_index=True, right_index=True)

# Dermatology
dermatology = fetch_ucirepo(id=33)
df = dermatology.data.features
dt = dermatology.data.targets
dermatology_df = pd.merge(df, dt, left_index=True, right_index=True)

# Mushroom
mushroom = fetch_ucirepo(id=73)
mf = mushroom.data.features
mt = mushroom.data.targets
mushroom_df = pd.merge(mf, mt, left_index=True, right_index=True)

# Iris
iris = fetch_ucirepo(id=53)
if_ = iris.data.features
it_ = iris.data.targets
iris_df = pd.merge(if_, it_, left_index=True, right_index=True)

soybean_df = soybean_df.dropna()
zoo_df = zoo_df.dropna()
heart_disease_df = heart_disease_df.dropna()
dermatology_df = dermatology_df.dropna()
breast_cancer_df = breast_cancer_df.dropna()
mushroom_df = mushroom_df.dropna()
iris_df = iris_df.dropna()

### 3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.

In [28]:
# Modification for Stochastic EM Algorithm
class BernoulliMixtureSEM:
    
    def __init__(self, n_components, max_iter, batch_size=100, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.tol = tol
    
    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            self.get_Neff()
            self.get_mu(self.x)
            self.get_pi()
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def iterate_batches(self):
        n_samples = len(self.x)
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        for start_idx in range(0, n_samples, self.batch_size):
            end_idx = min(start_idx + self.batch_size, n_samples)
            batch_indices = indices[start_idx:end_idx]
            yield self.x.iloc[batch_indices]

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
        self.old_mu = None
        self.old_pi = None
        self.old_gamma = None
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        epsilon = 1e-15
        mu_place = np.clip(mu, epsilon, 1 - epsilon)
        return np.tensordot(x, np.log(mu_place), (1,1))

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self, batch):
        self.mu = np.einsum('ik,id -> kd', self.gamma, batch) / self.Neff[:, None]
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)


### 5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.

### 6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.

In [32]:
def encode_categorical(df):
    encoder = LabelEncoder()
    encoded_df = df.copy()
    for column in df.columns:
        if df[column].dtype == 'object':
            encoded_df[column] = encoder.fit_transform(df[column])
    return encoded_df

datasets = ["soybean_df", "zoo_df", "heart_disease_df", "dermatology_df", "breast_cancer_df", "mushroom_df", "iris_df"]
encoded_datasets = {}

for dataset_name in datasets:
    df = globals()[dataset_name].copy()
    encoded_datasets[dataset_name] = encode_categorical(df)

def evaluate_clustering(dataset_name, algorithm, **kwargs):
    dataset = encoded_datasets[dataset_name]
    X = dataset.iloc[:, :-1]
    true_labels = dataset.iloc[:, -1]
    
    if algorithm == 'BernoulliMixtureSEM':
        model = BernoulliMixtureSEM(**kwargs)
        model.fit(X)
        labels = model.predict(X)
    elif algorithm == 'KModes':
        km = KModes(**kwargs)
        labels = km.fit_predict(X)
    else:
        raise ValueError("Algorithm not supported.")
    
    fmi, ari, nmi = None, None, None
    
    if true_labels is not None:
        fmi = fowlkes_mallows_score(true_labels, labels)
        ari = adjusted_rand_score(true_labels, labels)
        nmi = normalized_mutual_info_score(true_labels, labels)
    
    return fmi, ari, nmi

results = {}
algorithms = ['BernoulliMixtureSEM', 'KModes']
for dataset_name in datasets:
    results[dataset_name] = {}
    for algorithm in algorithms:
        if algorithm == 'BernoulliMixtureSEM':
            kwargs = {'n_components': 2, 'max_iter': 100}
        elif algorithm == 'KModes':
            kwargs = {'n_clusters': 2, 'max_iter': 100}
        fmi, ari, nmi = evaluate_clustering(dataset_name, algorithm, **kwargs)
        results[dataset_name][algorithm] = {'FMI': fmi, 'ARI': ari, 'NMI': nmi}

data = []

for dataset_name, algorithms in results.items():
    for algorithm, metrics in algorithms.items():
        data.append([dataset_name, algorithm, metrics['FMI'], metrics['ARI'], metrics['NMI']])

results_df = pd.DataFrame(data, columns=['Dataset', 'Algorithm', 'FMI', 'ARI', 'NMI'])
results_df

,Dataset,Algorithm,FMI,ARI,NMI
0,soybean_df,BernoulliMixtureSEM,0.617376,0.296578,0.552626
1,soybean_df,KModes,0.617376,0.296578,0.552626
2,zoo_df,BernoulliMixtureSEM,0.674122,0.447982,0.579111
3,zoo_df,KModes,0.674122,0.447982,0.579111
4,heart_disease_df,BernoulliMixtureSEM,0.431413,0.015681,0.007409
5,heart_disease_df,KModes,0.603760,0.296890,0.207837
6,dermatology_df,BernoulliMixtureSEM,0.542416,0.210019,0.456664
7,dermatology_df,KModes,0.479534,0.154536,0.362217
8,breast_cancer_df,BernoulliMixtureSEM,0.737819,0.000000,0.000000
9,breast_cancer_df,KModes,0.862549,0.678265,0.587960


### 7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
### 8. Write the report in Latex, push to your github with the codes.

In this report, we compare the performance of two clustering algorithms, BernoulliMixtureSEM and KModes, on various datasets using three evaluation metrics: Fowlkes-Mallows Index (FMI), Adjusted Rand Index (ARI), and Normalized Mutual Information (NMI). The datasets used are soybean_df, zoo_df, heart_disease_df, dermatology_df, breast_cancer_df, mushroom_df, and iris_df.


### Fowlkes-Mallows Index (FMI)

The FMI is a measure of similarity between two clusterings, with a value ranging from 0 to 1, where 1 indicates a perfect match. In our results, we observe that the FMI values are generally higher than the ARI and NMI values for both algorithms across most datasets.
For the soybean_df and zoo_df datasets, both algorithms achieve the same FMI, ARI, and NMI scores, indicating similar clustering performance. However, for the heart_disease_df dataset, KModes outperforms BernoulliMixtureSEM in terms of FMI, while BernoulliMixtureSEM has a higher FMI for the dermatology_df dataset.

In the case of the breast_cancer_df dataset, KModes achieves a higher FMI of 0.862549 compared to 0.737819 for BernoulliMixtureSEM. The opposite is true for the mushroom_df dataset, where BernoulliMixtureSEM has a lower FMI of 0.516542, while KModes has a higher FMI of 0.786584.
For the iris_df dataset, KModes has a slightly lower FMI of 0.528764 compared to 0.573462 for BernoulliMixtureSEM.

### Adjusted Rand Index (ARI)

The ARI is a measure of similarity between two clusterings, with a value ranging from -1 to 1, where 1 indicates a perfect match, 0 indicates a random labeling, and negative values indicate an anti-correlation.
In our results, we observe that the ARI values are generally lower than the FMI values for both algorithms across most datasets. For the soybean_df and zoo_df datasets, both algorithms achieve the same ARI scores of 0.296578 and 0.447982, respectively.

However, for the heart_disease_df dataset, KModes has a higher ARI of 0.296890 compared to 0.015681 for BernoulliMixtureSEM. Similarly, for the dermatology_df dataset, KModes has a higher ARI of 0.154536 compared to 0.210019 for BernoulliMixtureSEM.

In the case of the breast_cancer_df dataset, BernoulliMixtureSEM has an ARI of 0.000000, indicating a random labeling, while KModes achieves a higher ARI of 0.678265. For the mushroom_df dataset, KModes has a higher ARI of 0.488541 compared to 0.000690 for BernoulliMixtureSEM.

For the iris_df dataset, KModes has a higher ARI of 0.071126 compared to 0.000000 for BernoulliMixtureSEM, indicating a random labeling.


### Normalized Mutual Information (NMI)

The NMI is a measure of similarity between two clusterings, with a value ranging from 0 to 1, where 1 indicates a perfect match, and 0 indicates that the clusterings are independent.

In our results, we observe that the NMI values are generally lower than the FMI and ARI values for both algorithms across most datasets. For the soybean_df and zoo_df datasets, both algorithms achieve the same NMI scores of 0.552626 and 0.579111, respectively.

However, for the heart_disease_df dataset, KModes has a higher NMI of 0.207837 compared to 0.007409 for BernoulliMixtureSEM. Similarly, for the dermatology_df dataset, BernoulliMixtureSEM has a higher NMI of 0.456664 compared to 0.362217 for KModes.

In the case of the breast_cancer_df dataset, BernoulliMixtureSEM has an NMI of 0.000000, indicating independent clusterings, while KModes achieves a higher NMI of 0.587960. For the mushroom_df dataset, KModes has a higher NMI of 0.457618 compared to 0.002964 for BernoulliMixtureSEM.

For the iris_df dataset, KModes has a higher NMI of 0.149062 compared to 0.000000 for BernoulliMixtureSEM, indicating independent clusterings.


### Observations and Conclusions

Based on the results, we can make the following observations and conclusions:
1. The FMI values are generally higher than the ARI and NMI values for both algorithms across most datasets. This could be due to the different properties and assumptions of these evaluation metrics.
2. For the soybean_df and zoo_df datasets, both algorithms perform similarly, achieving the same scores across all three evaluation metrics.
3. For the heart_disease_df dataset, KModes outperforms BernoulliMixtureSEM in terms of FMI and ARI, but BernoulliMixtureSEM has a higher NMI.
4. For the dermatology_df dataset, BernoulliMixtureSEM has a higher FMI and NMI, while KModes has a higher ARI.
5. For the breast_cancer_df dataset, KModes outperforms BernoulliMixtureSEM across all three evaluation metrics.
6. For the mushroom_df dataset, KModes outperforms BernoulliMixtureSEM in terms of FMI, ARI, and NMI.
7. For the iris_df dataset, BernoulliMixtureSEM has a higher FMI, while KModes has higher ARI and NMI scores.
8. The differences in performance between BernoulliMixtureSEM and KModes can be attributed to the underlying assumptions and methodologies of these algorithms. BernoulliMixtureSEM is based on a mixture model and assumes a specific distribution for the data, while KModes is a partitioning-based algorithm specifically designed for categorical data.

In summary, the choice of the most appropriate clustering algorithm and evaluation metric depends on the characteristics of the dataset and the specific requirements of the problem at hand. It is essential to consider the strengths and limitations of each algorithm and evaluation metric when interpreting the results.